# Imports

In [1]:
import numpy as np
import pandas as pd
import os

/var/folders/z_/7fqm4zqs11zdv7cm41c6w_pc0000gn/T/ipykernel_3842/3339900087.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Wheter to generate videos or 3D images

In [2]:
generate_video = True
video_length = 10

# Read usable data
Read the usables.csv file, and filter out items that are not downloaded, and thus unavailable.

In [3]:
df = pd.read_csv("/Users/balazsmorvay/PycharmProjects/VAE/Data/usables.csv", sep=';')
for i in range(len(df)):
    filename = df.loc[i, 'FILE_ID']
    filepath = os.path.join('/Users/balazsmorvay/Downloads/ABIDE/data/Outputs/ccs/filt_noglobal/func_preproc',
                            f'{filename}_func_preproc.nii.gz')
    df.loc[i, "EXISTS"] = 1 if os.path.exists(filepath) else 0
df = df[df.EXISTS == 1]
df.head()

,FILE_ID,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,HANDEDNESS_CATEGORY,SITE_ID,EXISTS
1,Pitt_0050004,1,1,1,19.09,R,PITT,1
2,Pitt_0050005,1,1,2,13.73,R,PITT,1
3,Pitt_0050006,1,1,1,13.37,L,PITT,1
4,Pitt_0050007,1,1,1,17.78,R,PITT,1
5,Pitt_0050008,1,1,1,32.45,R,PITT,1


# Check number of data points in sites

In [4]:
df.drop(df[df.FILE_ID == "UM_1_0050302"].index, inplace=True) # drop this file, as it cannot be opened

In [5]:
df['SITE_ID'].value_counts()

SITE_ID
NYU         169
UM_1         80
USM          60
UCLA_1       55
YALE         47
TRINITY      44
MAX_MUN      42
PITT         41
KKI          39
STANFORD     33
SDSU         31
UM_2         31
LEUVEN_1     28
LEUVEN_2     26
OLIN         25
SBL          23
OHSU         22
UCLA_2       20
CALTECH      15
CMU           2
Name: count, dtype: int64

# Select the highest, and create train, test and val.csv files

In [6]:
df = df[df.SITE_ID == 'NYU']

if generate_video:
    start_times_series = pd.Series(np.arange(start=0, stop=161, step=10), name='START_TIME')
    df = df.merge(start_times_series, how='cross')
    df['END_TIME'] = df.apply(lambda x: x['START_TIME'] + video_length, axis=1)
    print(df.head())
else:
    timesteps = np.arange(0, 175)
    df = df.merge(pd.Series(timesteps, name='TIME_SLICE'), how='cross')

train_df = df.sample(frac=0.8,random_state=42)
remaining = df.drop(train_df.index)
val_df = remaining.sample(frac=0.5, random_state=42)
test_df = remaining.drop(val_df.index)

       FILE_ID  DX_GROUP  DSM_IV_TR  SEX  AGE_AT_SCAN HANDEDNESS_CATEGORY  \
0  NYU_0050954         1          1    2        14.75                 NaN   
1  NYU_0050954         1          1    2        14.75                 NaN   
2  NYU_0050954         1          1    2        14.75                 NaN   
3  NYU_0050954         1          1    2        14.75                 NaN   
4  NYU_0050954         1          1    2        14.75                 NaN   

  SITE_ID  EXISTS  START_TIME  END_TIME  
0     NYU       1           0        10  
1     NYU       1          10        20  
2     NYU       1          20        30  
3     NYU       1          30        40  
4     NYU       1          40        50  


In [7]:
train_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,START_TIME,END_TIME
count,2298.000000,2298.000000,2298.000000,2298.000000,2298.0,2298.000000,2298.000000
mean,1.575718,0.583551,1.210618,15.259160,1.0,80.274151,90.274151
std,0.494341,0.780827,0.407836,6.550382,0.0,49.035660,49.035660
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000,10.000000
25%,1.000000,0.000000,1.000000,10.480000,1.0,40.000000,50.000000
50%,2.000000,0.000000,1.000000,13.628000,1.0,80.000000,90.000000
75%,2.000000,1.000000,1.000000,19.228000,1.0,120.000000,130.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,160.000000,170.000000


In [8]:
val_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,START_TIME,END_TIME
count,288.000000,288.000000,288.000000,288.000000,288.0,288.000000,288.000000
mean,1.541667,0.597222,1.215278,16.175927,1.0,78.402778,88.402778
std,0.499128,0.740274,0.411730,6.832882,0.0,47.694779,47.694779
min,1.000000,0.000000,1.000000,7.150000,1.0,0.000000,10.000000
25%,1.000000,0.000000,1.000000,10.827500,1.0,40.000000,50.000000
50%,2.000000,0.000000,1.000000,14.210000,1.0,80.000000,90.000000
75%,2.000000,1.000000,1.000000,20.752500,1.0,120.000000,130.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,160.000000,170.000000


In [9]:
test_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,START_TIME,END_TIME
count,287.000000,287.000000,287.000000,287.000000,287.0,287.000000,287.000000
mean,1.592334,0.533101,1.170732,15.687056,1.0,79.407666,89.407666
std,0.492259,0.751305,0.376932,6.736877,0.0,50.111739,50.111739
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000,10.000000
25%,1.000000,0.000000,1.000000,10.690000,1.0,40.000000,50.000000
50%,2.000000,0.000000,1.000000,14.360000,1.0,80.000000,90.000000
75%,2.000000,1.000000,1.000000,19.228000,1.0,120.000000,130.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,160.000000,170.000000


In [10]:
train_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE_video/train.csv')
val_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE_video/val.csv')
test_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE_video/test.csv')

# Select the second highest to be the out of distribution test set

In [6]:
df = pd.read_csv("/Users/balazsmorvay/PycharmProjects/VAE/Data/usables.csv", sep=';')
for i in range(len(df)):
    filename = df.loc[i, 'FILE_ID']
    filepath = os.path.join('/Users/balazsmorvay/Downloads/ABIDE/data/Outputs/ccs/filt_noglobal/func_preproc',
                            f'{filename}_func_preproc.nii.gz')
    df.loc[i, "EXISTS"] = 1 if os.path.exists(filepath) else 0
df = df[df.EXISTS == 1]
df.head()

,FILE_ID,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,HANDEDNESS_CATEGORY,SITE_ID,EXISTS
1,Pitt_0050004,1,1,1,19.09,R,PITT,1
2,Pitt_0050005,1,1,2,13.73,R,PITT,1
3,Pitt_0050006,1,1,1,13.37,L,PITT,1
4,Pitt_0050007,1,1,1,17.78,R,PITT,1
5,Pitt_0050008,1,1,1,32.45,R,PITT,1


In [7]:
df.drop(df[df.FILE_ID == "UM_1_0050302"].index, inplace=True) # drop this file, as it cannot be opened
df['SITE_ID'].value_counts()

SITE_ID
NYU         169
UM_1         80
USM          60
UCLA_1       55
YALE         47
TRINITY      44
MAX_MUN      42
PITT         41
KKI          39
STANFORD     33
SDSU         31
UM_2         31
LEUVEN_1     28
LEUVEN_2     26
OLIN         25
SBL          23
OHSU         22
UCLA_2       20
CALTECH      15
CMU           2
Name: count, dtype: int64

In [8]:
um_df = df[df.SITE_ID == 'UM_1']
um_df.head()

,FILE_ID,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,HANDEDNESS_CATEGORY,SITE_ID,EXISTS
179,UM_1_0050272,1,1,1,14.2,R,UM_1,1
180,UM_1_0050273,1,1,1,16.8,R,UM_1,1
181,UM_1_0050274,1,1,1,14.2,R,UM_1,1
182,UM_1_0050275,1,1,1,11.5,L,UM_1,1
183,UM_1_0050276,1,1,2,16.8,-9999,UM_1,1


In [9]:
if generate_video:
    start_times_series = pd.Series(np.arange(start=0, stop=281, step=10), name='START_TIME')
    um_df = um_df.merge(start_times_series, how='cross')
    um_df['END_TIME'] = um_df.apply(lambda x: x['START_TIME'] + video_length, axis=1)
else:
    timesteps = np.arange(0, 295)
    um_df = um_df.merge(pd.Series(timesteps, name='TIME_SLICE'), how='cross')

In [10]:
um_df.head()

,FILE_ID,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,HANDEDNESS_CATEGORY,SITE_ID,EXISTS,START_TIME,END_TIME
0,UM_1_0050272,1,1,1,14.2,R,UM_1,1,0,10
1,UM_1_0050272,1,1,1,14.2,R,UM_1,1,10,20
2,UM_1_0050272,1,1,1,14.2,R,UM_1,1,20,30
3,UM_1_0050272,1,1,1,14.2,R,UM_1,1,30,40
4,UM_1_0050272,1,1,1,14.2,R,UM_1,1,40,50


In [11]:
um_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,START_TIME,END_TIME
count,2320.00000,2320.000000,2320.000000,2320.000000,2320.0,2320.00000,2320.00000
mean,1.57500,-124.487500,1.275000,13.913750,1.0,140.00000,150.00000
std,0.49445,1111.209166,0.446611,2.866876,0.0,83.68404,83.68404
min,1.00000,-9999.000000,1.000000,8.200000,1.0,0.00000,10.00000
25%,1.00000,0.000000,1.000000,11.500000,1.0,70.00000,80.00000
50%,2.00000,0.000000,1.000000,13.850000,1.0,140.00000,150.00000
75%,2.00000,1.000000,2.000000,16.100000,1.0,210.00000,220.00000
max,2.00000,3.000000,2.000000,19.200000,1.0,280.00000,290.00000


In [12]:
um_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE_video/UM_1.csv')